# INF 8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2018 - TP1 - Méthodes de recherche 
### Membres de l'équipe
    - Membre 1
    - Membre 2
    - Membre 3



## LE VÉLO À MONTRÉAL
Chaque année, Montréal accueille à peu près 10 millions de touristes. Soucieuse de la qualité de leur séjour, Tourisme Montréal a entamé un projet de développement d’une nouvelle application mobile afin d’assister les touristes lors de leurs déplacements dans la ville. Cette application a pour but d’aider l’utilisateur à planifier sa visite des importantes attractions de la ville, de la façon la plus efficace possible (ie, sur la durée la plus courte). Étant donné qu’il a été observé que le moyen de transport privilégié des touristes pour explorer Montréal est le vélo, cette application a pour but de générer des circuits cyclables de durée minimale. Plus précisément, étant donné une liste d’attractions munie de points de départ et d’arrivée, la tâche est de proposer, à chaque fois, un chemin qui passe par toutes les attractions indiquées une seule fois, qui débute au point de départ et qui s’achève au point d’arrivée et dont la durée de trajet est minimale.

<img src="images/montreal.png" alt="" width="800"/>

Le travail demandé dans ce TP est de développer l’algorithme interne de l’application. Nous explorerons trois mécanismes de résolution différents :
1. Définition et exploration naïve d’un arbre de recherche
2. Exploration plus efficace en utilisant l’algorithme A*
3. Optimisation locale en utilisant une métaheuristique de recherche à voisinage variable (Variable Neighborhood Search, VNS)

## PRÉSENTATION DU PROBLÈME
Une façon naturelle de représenter notre problème est d’utiliser un graphe $G=(V, A)$ dirigé et complet. Chaque sommet dans $V$ est une attraction donnée et chaque arc dans $A$ représente une piste cyclable entre deux attractions distinctes. Chaque paire de sommets $i$ et $j$ est reliée par une paire d’arcs $a_{ij}$ et $a_{ji}$ dont les poids respectifs $w(a_{ij})$ et $w(a_{ji})$ ne sont pas nécessairement égaux. Concrètement, ces poids représentent la durée du trajet d’un sommet à l’autre (ainsi, $w$ est telle que $w : A \to\mathbb R^+$).

La liste des attractions à visiter est indiquée comme la suite $P = (p_1, ..., p_m)$ où $p_1$ et $p_m$ sont les sommets de départ et d’arrivée, respectivement.

## 1. DÉFINITION ET EXPLORATION NAÏVE D’UN ARBRE DE RECHERCHE (5 points)
Définissons un arbre de recherche $\mathcal{T}$ où chaque nœud représente une solution partielle $S$. Soient $V(S) \subseteq V$ et $A(S) \subset A$ l’ensemble des sommets visités et l’ensemble des arêtes sélectionnées, respectivement. Ainsi, le coût d’une solution est donné par :
$$g(S) = \sum_{a \in A(S)} w(a)$$

Seule l’origine est visitée initialement. Ainsi, la racine de l’arbre de recherche contient une solution partielle vide $S_{\textrm{root}}$ telle que $V(S_{\textrm{root}})=\{p_1\}$ et $A(S_{\textrm{root}}) = \emptyset$.

<img src="images/tree1.png" alt="" width="100"/>

À la suite de cela, les nœuds subséquents dans l’arbre sont tous créés en ajoutant, à chaque solution partielle $S$, un sommet subséquent dans $P\backslash V(S)$ avec l’arc correspondant dans $A$ qui relie ce sommet à la dernière attraction visitée. Le sommet $p_m$ n’est ajouté qu’à la fin, lorsqu’il est le seul sommet non encore visité. Plus formellement, si on note le sommet à ajouter $c$ et le dernier sommet visité $c'$, alors la nouvelle solution partielle obtenue est $V(S) \gets V(S) \cup \{c\}$ et $A(S) \gets A(S) \cup \{(c’,c)\}$.

Ci-dessous est un exemple de l’arbre étendu depuis sa racine où $c'$ = $p_1$ :

<img src="images/tree2.png" alt="" width="400"/>

À la fin, les feuilles de l’arbre sont des solutions complètes :

<img src="images/tree3.png" alt="" width="600"/>

### 1.1 Code
La fonction fournie ci-dessous permet d’extraire d’un fichier un graphe qui répond aux spécifications détaillées plus haut. Cette fonction retourne une $\texttt{ndarray}$ ($\texttt{graph}$) de taille $|V|\times |V|$ où $\texttt{graph[i,j]}$ représente le temps nécessaire pour traverser la piste cyclable de $i$ vers $j$.



In [1]:
import numpy as np

def read_graph():
    return np.loadtxt("montreal", dtype='i', delimiter=',')

graph = read_graph()

Notre première tâche est de définir la classe qui représente une solution partielle. Son constructeur est donné et reçoit comme argument la liste des sommets (attractions $P$) à visiter et le graphe ($G$). Celui-ci crée la solution $S_{\textrm{root}}$ avec les attributs suivants :
- $\texttt{g}$ : le coût de la solution partielle
- $\texttt{visited}$ : représente $V(S)$, discuté plus haut. Par définition, $\mathtt{vistited[-1]}$ représente le dernier sommet ajouté, $ c $.
- $\texttt{not}\_\texttt{visited}$ : représente $P\backslash V(S)$
- $\texttt{graph}$: représente le graphe G

Ensuite, il est demandé d’implanter la méthode $\texttt{add}$ qui mets à jour la solution partielle en ajoutant une nouvelle attraction à visiter parmi la liste $\texttt{not}\_\texttt{visited}$. Cette méthode reçoit comme arguments l’index du sommet à visiter parmi $\texttt{not}\_\texttt{visited}$ ainsi que le graphe courant.

Implantez $\texttt{add}$ :

In [2]:
import copy

class Solution:
    def __init__(self, places, graph):
        """
        places: a list containing the indices of attractions to visit
        p1 = places[0]
        pm = places[-1]
        """
        self.g = 0 # current cost
        self.graph = graph
        self.visited = [places[0]] # list of already visited attractions
        self.not_visited = copy.deepcopy(places[1:]) # list of attractions not yet visited
    
    def __𝚕𝚝__(self, other):
        return other > self.g

    def __le__(self,other):
        return(self.g<=other)

    def __gt__(self,other):
        return(self.g>other)

    def __ge__(self,other):
        return(self.g>=other)

    def __eq__(self,other):
        return (self.g==other)
        
    def add(self, idx):
        """
        Adds the point in position idx of not_visited list to the solution
        """
        self.visited.append(idx)
        self.not_visited.remove(idx)
        self.g += self.graph[self.visited[-2],idx]
        
    def swap(self, idx_1, idx_2):
        if idx_1>idx_2:
            idx_1,idx_2=idx_2,idx_1
        v = self.visited
        place_1 = v[idx_1]
        place_2 = v[idx_2]
        if abs(idx_1-idx_2)>1:
            weight_1 =  self.graph[v[idx_1-1],place_1]
            weight_2 =  self.graph[place_1,v[idx_1+1]]
            weight_3 =  self.graph[v[idx_2-1],place_2]
            weight_4 =  self.graph[place_2,v[idx_2+1]]    
            self.g -= (weight_1 + weight_2 + weight_3 + weight_4)
            weight_1 =  self.graph[v[idx_1-1],place_2]
            weight_2 =  self.graph[place_2,v[idx_1+1]]
            weight_3 =  self.graph[v[idx_2-1],place_1]
            weight_4 =  self.graph[place_1,v[idx_2+1]]    
            self.g += (weight_1 + weight_2 + weight_3 + weight_4)
            self.visited[idx_1] = place_2
            self.visited[idx_2] = place_1
        else:
            weight_1 =  self.graph[v[idx_1-1],place_1]
            weight_2 =  self.graph[place_1,place_2]
            weight_3 =  self.graph[place_2,v[idx_2+1]]
            self.g -= (weight_1 + weight_2 + weight_3)
            weight_1 =  self.graph[v[idx_1-1],place_2]
            weight_2 =  self.graph[place_2,place_1]
            weight_3 =  self.graph[place_1,v[idx_2+1]]
            self.g += (weight_1 + weight_2 + weight_3)
            self.visited[idx_1] = place_2
            self.visited[idx_2] = place_1            


La prochaine étape est d’implanter une stratégie de parcours de l’arbre de recherche. Une première méthode simple est naïve est de mettre en œuvre une recherche en largeur ([Breadth-first search](https://moodle.polymtl.ca/pluginfile.php/444662/mod_resource/content/1/recherche_en_largeur.mp4), BFS).

Implantez $\texttt{bfs}$ qui mets en œuvre cette recherche. Elle prend en arguments le graphe courant ainsi que la liste des attractions à visiter $P$ et elle retourne la meilleure solution trouvée.

In [3]:
from queue import Queue

def bfs(graph, places):
    """
    Returns the best solution which spans over all attractions indicated in 'places'
    """
    n = len(places) - 1
    solutions = Queue() # cree une file contenant les solutions visites
    sol = Solution(places=places, graph=graph) # cree la solution initale
    solutions.put(sol)  # la solution initiale est place dans la file
    for i in range(n):  # pour les places restant a visiter
        if i != n-1:    # s il reste plus qu une place a visiter
            for j in range(solutions.qsize()): # on effectue la recherche pour les successeurs non visites
                sol_j = solutions.get()        # j_eme solution
                not_visited = sol_j.not_visited[:-1]
                for k in not_visited:
                    temp = copy.deepcopy(sol_j)
                    temp.add(k)
                    solutions.put(temp)
        else: # s il reste plus une place a visiter
            for j in range(solutions.qsize()):
                sol_j = solutions.get() # j_eme solution
                temp = copy.deepcopy(sol_j)
                temp.add(places[-1])
                solutions.put(temp)
    # on retourne la solution de cout minimale
    min_cost = float('Inf')
    min_sol = None
    for i in range(solutions.qsize()):
        sol_i = solutions.get()
        if min_cost > sol_i:
            min_cost = sol_i.g
            min_sol = sol_i
    return min_sol
    # retourne le nombre de noeuds explores
    # total de larbre
    
    
    
    

### 1.2 Expérimentations

On propose trois exemples d’illustration pour tester notre recherche en largeur. Le premier exemple prend en compte 7 attractions, le second 10 et le dernier 11. Vu que cette recherche énumère toutes les solutions possibles, le troisième exemple risque de prendre un temps considérable à s’achever.

Mettez en œuvre ces expériences et notez le nombre de nœuds explorés ainsi que le temps de calcul requis.

In [4]:
import time 

#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

27
--- 0.0200042724609375 seconds ---


In [5]:
#test 2 -------------- OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

30
--- 6.506963491439819 seconds ---


In [6]:
#test 3 -------------- OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

26
--- 58.52444863319397 seconds ---


## 2. RECHERCHE GUIDÉE À L’AIDE DE L’ALGORITHME A\* (7.5 points)
Pour notre deuxième méthode de recherche, au lieu d’énumérer toutes les solutions possibles, nous effectuons une recherche guidée à l’aide de l’algorithme A\*. Comme vu en classe, A\* est une recherche où les nœuds à explorer sont priorisés en fonction du coût courant d’une solution $g(S)$ ainsi que d’une estimation du coût restant vers la solution finale donné par une heuristique $h(S)$.

Dans le cas d’une minimisation, $h(S)$ est une borne inférieure du coût réel restant et on priorise l’exploration des nœuds dont $f(S) = g(S)+h(S)$ est le plus petit. Avec cette méthode, la première solution complète trouvée est assurément la solution optimale.

Pour une solution donnée $S$ avec un dernier sommet visité $c$, une possible fonction $h$ est telle que :

$h(S) =$ Le poids du chemin le plus court entre $c$ et $p_m$ dans le sous graphe $G_S$ contenant les sommets $P\backslash V(S) \cup \{c\}$

Remarque que ce chemin le plus court utilisé dans le calcul de l’estimation $h$ entre l’attraction courante et l’arrivée ne passera pas nécessairement pas tous les sommets restants.


Notre algorithme A\* se présente comme ceci :
1. Définir l’arbre de recherche $\mathcal{T}$ exactement comme auparavant. Le calcul de $h$ pour la solution initiale est inutile : c’est la seule solution qu’on a.
2. Sélectionner le meilleur nœud candidat pour expansion. La solution partielle $S_b$ de ce nœud candidat est telle que :

   $$ f(S_b) \leq f(S) \quad \forall S \in \mathcal{T} \qquad S_B, S \text{ pas encore sélectionnés}$$

   Si $S_b$ est une solution complète, l’algorithme s’arrête et $S_b$ est assurément la solution optimale, sinon on continue à l’étape 3.
3. Créer des solutions subséquentes qui connectent la dernière attraction visitée à chacune des attractions restantes. Attention, on ignore l’arrivée tant que celle-ci n’est pas la seule qui reste.
 - Mettez à jour les listes des sommets visités et non visités
 - Calculez $g$ et $h$ pour chaque solution
 - Insérer la nouvelle solution partielle dans l’arbre.
4. Répéter 2 et 3.


### 2.1 Code
Commençons d’abord par compléter la classe $\texttt{Solution}$ pour prendre en compte les changements nécessaires à A\* (on a besoin notamment d’un attribut supplémentaire pour l’estimation $h$).

On verra plus tard que A\* s’implante à l’aide d’une file de priorité (priority queue). Pour que celle-ci marche, il est nécessaire de surcharger (overload) l’opérateur de comparaison « < » relatif à nos objets $\texttt{Solution}$. En sachant ce qui fait qu'une solution est meilleure qu’une autre pour l'exploration, implanter la méthode $\_\_\texttt{lt}\_\_$ dans $\texttt{Solution}$. Son prototype est $\_\_\texttt{lt}\_\_\texttt{(self, other)}$.

Maintenant, nous devons implanter la fonction d’estimation $h$. Pour cela, on utilise l’[algorithme de Dijkstra](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) pour trouver le chemin le plus court entre la dernière attraction visitée $c$ et l’arrivée $p_m$. Il est possible d’adapter cet algorithme pour qu’il s’arrête dès que le chemin le plus court entre c et pm est trouvé.

**Prescriptions d’implantation :**
- Appliquer Dijkstra pour trouver le chemin le plus court entre $c$ et $p_m$
- Retourner le poids de ce chemin

In [7]:
class Vertex:
    """
    Une classe servant a representer les noeuds d un graphe
    """
    def __init__(self, place_id, dist=float('Inf'), prev= None):
        self.dist = dist # current cost
        self.prev = prev # noeud precedent
        self.place_id=place_id # numero de la place
        self.visited = False
        self.order = 0
        self.parent = None

    def  __𝚕𝚝__(self, other):
        return other > self.dist
     
    def __le__(self,other):
        return(self.dist<=other)

    def __gt__(self,other):
        return(self.dist>other)

    def __ge__(self,other):
        return(self.dist>=other)

    def __eq__(self,other):
        return (self.dist==other)
    
    def __hash__(self):
        return hash(str(self.place_id))

    def is_visited(self):
        return self.visited
    
    def set_visited(self):
        self.visited = True

    def set_order(self, n):
        self.order = n
    
def fastest_path_estimation(sol):
    """
    Returns the time spent on the fastest path between 
    the current vertex c and the ending vertex pm
    """
    c = sol.visited[-1]      # recolte le noeud c
    pm = sol.not_visited[-1] # recolte le noeud pm
    v_list=list()            # cree un liste
    c_vertex =Vertex(c,0)    # cree le noeud c
    v_list.append(c_vertex)  # ajoute le noeud c a la liste
    graph = sol.graph
    for i in sol.not_visited[:-1]: # on ajoute les noeuds non visite a la liste
        v_list.append(Vertex(i))
    pm_vertex=Vertex(pm)           # on ajoute le noeud pm a la liste
    v_list.append(pm_vertex)
    
    while v_list:                  # tant qu il reste des elements dans la liste
        idx=v_list.index(min(v_list)) # on recolte l element de cout minimum
        u=v_list.pop(idx)             # on le retire de la liste
        
        # pour tous les elements de la liste on verifie s il y a un chemin plus
        # court a l aide du point u
        for v in v_list:
            alt=u.dist+graph[u.place_id,v.place_id]
           
            if alt < v: # si le chemin est plus court, on change son predecesseur et on ajuste le cout
                v.dist=alt
                v.prev=u

    path=[pm_vertex.place_id] # enleve la variable path?
    temp=pm_vertex
   
    while temp.prev != None:
        path.append(temp.prev.place_id)
        temp= temp.prev
    return path[::-1], pm_vertex.dist 
    
    

Finalement, il est temps d’implanter A\*. On aura besoin d’une file de priorité qui retournera toujours le meilleur nœud candidat de $\mathcal{T}$ pour l’étendre (l’opérateur surchargé de comparaison assure cela).

**Prescriptions d’implantation (cf. détail des étapes de l’algorithme plus haut) :**
- Tant que les solutions extraites de la file de priorité ne sont pas complètes :
  *	Sélectionner et étendre le nœud extrait de la file comme détaillé plus haut
  * Calculer $g$ et $h$ pour chaque nouvelle solution partielle obtenue
  * Remettre ces solutions dans la file
- Retourner la première solution complète extraite de la file (c’est la solution optimale)

In [12]:
import heapq

def A_star(graph, places):
    """
    Performs the A* algorithm
    """
    
    # blank solution
    root = Solution(graph=graph, places=places)
    
        
    # search tree T
    T = []
    heapq.heapify(T)
    heapq.heappush(T, (0,root))
    # tant que tous noeuds de la solution minimal na pas ete visite
    while heapq.nsmallest(1,T)[0][1].not_visited[:-1]:

        c_min=heapq.heappop(T)
        sol_min = c_min[1]

        # on recolte g
        c_g = sol_min.g
        # pour tous les voisins non visite
        for i in sol_min.not_visited[:-1]:
            # on cree une nouvelle solution possible avec ce voisin
            temp_sol = copy.deepcopy(sol_min)
            temp_sol.add(i)
            # on calcule le cout h
            f = temp_sol.g + fastest_path_estimation(temp_sol)[1]
            # on ajoute cette solution a l arbre
            heapq.heappush(T, (f, temp_sol))
    # on retourne la solution minimale
    last = heapq.nsmallest(1,T)[0][1]
    last.add(last.not_visited[-1])
    return last
        

### 2.2 Expérimentations

On ajoute un Quatrième exemple d’exécution avec 15 attractions. Là encore, mettez en œuvre ces expériences avec le nouvel algorithme A\* conçu et notez le nombre de nœuds explorés ainsi que le temps de calcul requis.

In [13]:
#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

27
[0, 5, 13, 16, 6, 9, 4]
--- 0.017003536224365234 seconds ---


In [14]:
#test 2  --------------  OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 0.415783166885376 seconds ---


In [15]:
#test 3  --------------  OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 1.7305529117584229 seconds ---


In [16]:
#test 4  --------------  OPT. SOL. = 40
start_time = time.time()
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: 

### 2.3 Une meilleure borne inférieure

Notre algorithme A\* est déjà beaucoup plus efficace qu’une recherche naïve. Cependant, la qualité de l’heuristique $h$ a un très grand impact sur la vitesse de A\*. Une heuristique plus serrée devrait accélérer A\* de façon significative. Notre estimation $h$ basée sur Dijkstra est très large à cause du fait qu’elle ne considère pas toutes les attractions restantes.

Une meilleure heuristique pourrait être basée sur la **Spanning Arborescence of Minimum Weight** qui s’apparente à une Minimum Spanning Tree pour graphes orientés. On propose de construire une telle Spanning Arborescence sur le reste des attractions $P\backslash V(S) \cup \{c\}$. Ici la racine est la dernière attraction visitée $c$. Une façon classique de résoudre ce problème est d’utiliser l’[algorithme de Edmonds](https://en.wikipedia.org/wiki/Edmonds%27_algorithm).

Implantez cet algorithme et refaites les expériences avec A\* en utilisant cette nouvelle heuristique :

In [145]:
class Edge:
    """
    Une classe servant a representer les arretes d un graphe
    """
    def __init__(self, start, end, weight):
        self.start = start # current cost
        self.end = end # noeud precedent
        self.weight=weight # numero de la place

    def  __𝚕𝚝__(self, other):
        return other > self.weight
     
    def __le__(self,other):
        return(self.weight<=other)

    def __gt__(self,other):
        return(self.weight>other)

    def __ge__(self,other):
        return(self.weight>=other)

    def __eq__(self,other):
        return (self.weight==other)

class Vertex_id:
    """
    Une classe servant a representer les noeuds d un graphe
    """
    def __init__(self, place_id, dist=float('Inf')):
        self.dist = dist # current cost
        self.place_id=place_id # numero de la place
        self.visited = False
        self.order = 0
        self.parent = None

    def  __𝚕𝚝__(self, other):
        return other > self.place_id
     
    def __le__(self,other):
        return(self.place_id<=other)

    def __gt__(self,other):
        return(self.place_id>other)

    def __ge__(self,other):
        return(self.place_id>=other)

    def __eq__(self,other):
        return (self.place_id==other)
    
    def __hash__(self):
        return hash(str(self.place_id))
    
    def __len__(self):
        return 1

    def __getitem__(self, i):
        return self
    
    def is_visited(self):
        return self.visited
    
    def set_visited(self):
        self.visited = True

    def set_order(self, n):
        self.order = n
       
    
class Graph():
    def __init__(self, vertices, edges=[]):
        self.v = vertices
        self.edges = edges
        self.graph = {}
        self.count = 1
        for i in vertices:
            self.graph[i] = []
        for i in edges:
            self.graph[i.start].append(i.end)
        
    def get_vertives(self):
        return self.v
    
    def get_neighbors(self, node):
        return self.graph[node]

def arborescence_contraction(v_list, graph, cycle_list=[]):
    edge_list = list()
    min_cycle_src = None
    min_cycle_dst = None
    min_w_cycle = float('Inf')
    test = False
    for j in v_list[1:]: # destination
        min_v = None
        min_w = float('Inf')
        for i in v_list: # source
            if i.place_id != j.place_id:
                if j not in cycle_list:
                    temp_w = graph[i.place_id, j.place_id]
                    if temp_w < min_w:
                        min_w = temp_w
                        min_v = i
                else:
                    if i not in cycle_list:
                        temp_w = graph[i.place_id, j.place_id]
                        if temp_w<min_w_cycle:
                            min_w_cycle = temp_w
                            min_cycle_src = i
                            min_cycle_dst = j
        if min_v is not None:
            print(min_v.place_id, j.place_id, min_w)
            edge_list.append(Edge(min_v,j,min_w))
            j.parent = min_v
    if min_cycle_src != None and min_cycle_dst != None:
        print(min_cycle_src.place_id, min_cycle_dst.place_id, min_w_cycle)
        # j.parent = min_v    
    return edge_list


def arborescence_contraction_1(v_list, graph):
    edge_list = list()
    min_cycle_src = None
    min_cycle_dst = None
    min_w_cycle = float('Inf')
    test = False
    for j in v_list[1:]: # destination
        min_v_dst = None
        min_v_src = None
        min_v_i = None
        min_w = float('Inf')
        for vertices_dst in j.place_id: # differente destination du noeud
            for i in v_list: # source
                for vertices_src in i.place_id: # differente source du noeud
                    if i != j:
                        if vertices_src != vertices_dst:
                            temp_w = graph[vertices_src, vertices_dst]
                            if temp_w < min_w:
                                min_w = temp_w
                                min_v_src = vertices_src
                                min_v_dst = vertices_dst
                                min_v_i = i

        if min_v_src is not None:
            print(min_v_src, min_v_dst, min_w)
            edge_list.append(Edge(min_v_src,min_v_dst,min_w))
            j.parent = min_v_i
            j.dist = min_w
    #if min_cycle_src != None and min_cycle_dst != None:
        #print(min_cycle_src.place_id, min_cycle_dst.place_id, min_w_cycle)
        # j.parent = min_v    
    return edge_list

def find_cycle(v_list):
    cycle_list = []
    v_list_len = len(v_list)
    cycle_id = []
    cycle_list = []
    test = True
    for i in v_list[1:]: # on recherche les cycles
        leaf = i
        parent = leaf.parent
        cycle = True
        for j in range(v_list_len):
            if parent == None:
                cycle = False
                break
            print(leaf)
            leaf = parent
            parent = leaf.parent
        if cycle:
            # trouver le cycle en faisant une iteration
            cycle_path = list()
            cycle_id = list()
            begin = leaf
            cycle_path.append(begin)
            leaf = leaf.parent
            parent = leaf.parent
            while begin.place_id != leaf.place_id:
                cycle_path.append(leaf)
                leaf = parent
                parent = leaf.parent
            for j in cycle_path:
                cycle_id.append(j.place_id)
            test = True
            for j in cycle_list:
                for k in j:
                    if k in cycle_path:
                        test = False
                        break
            if test:
                cycle_list.append(cycle_path)
    return cycle_list

def find_cycle_1(v_list):
    cycle_list = []
    v_list_len = len(v_list)
    cycle_id = []
    cycle_list = []
    test = True
    for i in v_list[1:]: # on recherche les cycles
        leaf = i
        parent = leaf.parent
        cycle = True
        for j in range(v_list_len):
            if parent == None:
                cycle = False
                break
            leaf = parent
            parent = leaf.parent
        if cycle:
            # trouver le cycle en faisant une iteration
            cycle_path = list()
            cycle_id = list()
            begin = leaf
            cycle_path.append(begin)
            leaf = leaf.parent
            parent = leaf.parent
            while begin.place_id != leaf.place_id:
                cycle_path.append(leaf)
                leaf = parent
                parent = leaf.parent
            for j in cycle_path:
                cycle_id.append(j.place_id)
            test = True
            for j in cycle_list:
                for k in j:
                    if k in cycle_path:
                        test = False
                        break
            if test:
                cycle_list.append(cycle_path)
    return cycle_list

def array_to_dict(places, graph):
    dict_init = {}
    for i in places:
        dict_init[i] = {}
        for j in places:
            dict_init[i][j] = graph[i][j]
    return dict_init

def minimum_spanning_arborescence(sol):
    """
    Returns the cost to reach the vertices in the unvisited list 
    """
    places = []
    c = sol.visited[-1]# recolte le noeud c
    places.append(c)
    pm = sol.not_visited[-1] # recolte le noeud pm
    v_list=list()            # cree un liste
    c_vertex =Vertex_id([c])    # cree le noeud c
    v_list.append(c_vertex)  # ajoute le noeud c a la liste
    for i in sol.not_visited[:-1]: # on ajoute les noeuds non visite a la liste
        places.append(i)
        v_list.append(Vertex_id([i]))
    pm_vertex=Vertex_id([pm])           # on ajoute le noeud pm a la liste
    places.append(pm)
    dicti_g = array_to_dict(places, sol.graph)
    print(dicti_g)
    v_list.append(pm_vertex)
    v_init = copy.deepcopy(v_list)
    v_list_list = []
    v_list_list.append(v_init)
    e_list_list = []
    new_g_list = []
    cycle_list_list = []
    len_cycle = 1
    while len_cycle != 0:
        ## cree un nouveau graph de la classe graph
        print('\nedge_list')
        edge_list = arborescence_contraction_1(v_list, sol.graph)
        e_list_list.append(copy.deepcopy(edge_list))
        print('\nv_list')
        print(v_list)
        cycle_list = find_cycle_1(v_list)
        cycle_list_list.append(copy.deepcopy(cycle_list))
        print('\ncycle_list')
        print(cycle_list)
        
        new_g = copy.deepcopy(sol.graph)
        for v in v_list:
            for cycle in cycle_list:
                for node in cycle:
                    for src_id in v.place_id:
                        for dst_id in node.place_id:
                            new_g[src_id,dst_id] -= node.dist
        sol.graph = new_g
        len_cycle = len(cycle_list)

        if len_cycle==0:
            break
        v_temp = copy.deepcopy(v_init[1:])        
        v_list = []
        v_list.append(Vertex_id([0]))
        for cycle in cycle_list:
            temp_list = []
            for node in cycle:
                for node_id in node.place_id:
                    temp_list.append(node_id)
                    v_temp.remove(Vertex_id([node_id]))

            v_list.append(Vertex_id(temp_list))
        for node in v_temp:
            v_list.append(Vertex_id(node.place_id))
        v_list_list.append(v_list)
        print('\nv_list')
        print(v_list)
        # update des poids de g
    print('\n\n\nDECONTRACTION')
    final_edge_list = []
    final_node_list = []
    for i in range(len(v_list_list)):
        v_list = v_list_list.pop()
        e_list = e_list_list.pop()
        print(v_list)
        print(e_list)
        for e in e_list:
            if e.end not in final_node_list:

    print(final_node_list)
    print(final_edge_list)
            

    return v_list, edge_list

    

In [146]:
    
    print(v_list)
    edge_list = arborescence_contraction_1(v_list, new_g)
    print(edge_list)
    for v in v_list:
        print(v.place_id)

    cycle_list = find_cycle_1(v_list)
    print('cycle_list')
    print(cycle_list)
    v_list = []
    v_list.append(Vertex_id([0]))
    for cycle in cycle_list:
        temp_list = []
        for node in cycle:
            for node_id in node.place_id:
                temp_list.append(node_id)
        v_list.append(Vertex_id(temp_list))
    print('v_list')
    print(v_list)
    edge_list = arborescence_contraction_1(v_list, new_g)
    cycle_list = find_cycle_1(v_list)
    print(len(cycle_list))

NameError: name 'v_list' is not defined

In [147]:
graph = read_graph()
places=[0, 13, 12, 11,14, 2, 3]
sol = Solution(places=places, graph=graph)
v, e = minimum_spanning_arborescence(sol)

{0: {0: 0, 13: 21, 12: 20, 11: 22, 14: 21, 2: 8, 3: 12}, 13: {0: 20, 13: 0, 12: 2, 11: 2, 14: 1, 2: 15, 3: 5}, 12: {0: 19, 13: 1, 12: 0, 11: 2, 14: 1, 2: 11, 3: 6}, 11: {0: 21, 13: 1, 12: 1, 11: 0, 14: 2, 2: 12, 3: 7}, 14: {0: 22, 13: 2, 12: 1, 11: 1, 14: 0, 2: 10, 3: 8}, 2: {0: 8, 13: 14, 12: 11, 11: 12, 14: 11, 2: 0, 3: 2}, 3: {0: 10, 13: 7, 12: 7, 11: 8, 14: 10, 2: 2, 3: 0}}

edge_list
12 13 1
11 12 1
14 11 1
13 14 1
3 2 2
2 3 2

v_list
[<__main__.Vertex_id object at 0x00000235F2F72748>, <__main__.Vertex_id object at 0x00000235F2F8CBA8>, <__main__.Vertex_id object at 0x00000235F2F8CA58>, <__main__.Vertex_id object at 0x00000235F2F8C208>, <__main__.Vertex_id object at 0x00000235F2F8CEF0>, <__main__.Vertex_id object at 0x00000235F2F8CF98>, <__main__.Vertex_id object at 0x00000235F2F8CC88>]

cycle_list
[[<__main__.Vertex_id object at 0x00000235F2F8CEF0>, <__main__.Vertex_id object at 0x00000235F2F8CBA8>, <__main__.Vertex_id object at 0x00000235F2F8CA58>, <__main__.Vertex_id object at 0

## 3. RECHERCHE LOCALE À VOISINAGE VARIABLE  (7.5 points)

Cette fois-ci, au lieu de construire une solution optimale depuis une solution vide, on commence d’une solution complète, non-optimale, qu’on améliore à l’aide d’une recherche locale en utilisant une recherche locale à voisinage variable ([Variable Neighborhood Search](https://en.wikipedia.org/wiki/Variable_neighborhood_search), VNS).

<img src="images/vns.png" alt="" width="800"/>

### 3.1 Code

On commence par créer une solution initiale. Celle-ci est une suite ordonnée des attractions de $p_1$ à $p_m$ dans $P$. Pour cela, on fait appel à une [recherche en profondeur (Depth-First Search, DFS)](https://moodle.polymtl.ca/pluginfile.php/445484/mod_resource/content/1/recherche_en_profondeur.mp4) qu’on arrête aussitôt qu’une solution complète est trouvée. Pour aider à diversifier la recherche, la méthode permettant de générer une solution initiale peut être randomisée de telle sorte que l'algorithme VNS puisse lancer la recherche dans différentes régions de l'espace solution. Ainsi, dans la fonction DFS, la sélection de l'enfant pour continuer la recherche doit être aléatoire.

**Prescriptions d’implantation :**
- Mettre en œuvre une recherche en profondeur
- Créer un objet $\texttt{Solution}$ relatif à cette solution
- Ajuster les attributs de cet objet avec les bonnes valeurs de coûts et d’attractions visitées
- Retourner la solution trouvée.

In [804]:
from random import shuffle, randint, sample

def initial_sol(graph, places):
    """
    Return a completed initial solution
    """
    sol = Solution(places=places, graph=graph)
    v_list = []
    for i in places:
        v_list.append(Vertex_id(i))
    e_list = []
    for v1 in v_list:
        for v2 in v_list:
            if v1 != v2:
                e = Edge(v1,v2, graph[v1.place_id,v2.place_id])
                e_list.append(e)
    g = Graph(v_list,e_list)
    dfs(g, sol)
    return sol


def dfs(G, sol):
    """
    Performs a Depth-First Search
    """
    for node in G.get_vertives():
        if not node.is_visited():
            dfs_visit(G, node, sol)   
    return


def dfs_visit(G, root, sol):
    pile = []
    pile.append(root)  # root devient racine d'une nouvelle arborescence.
    final = sol.not_visited[-1]
    while pile:
        u = pile.pop()
        if not u.is_visited():
            if u.place_id != 0:
                sol.add(u.place_id)
            u.set_visited()
            u.set_order(G.count)
            G.count += 1
        for neighbor in G.get_neighbors(u):
            if not neighbor.is_visited():
                if neighbor != final:
                    pile.append(neighbor)
            pile = sample(pile, len(pile))

    return

In [274]:
places=[0, 13, 12, 11, 14, 2, 3]
init_sol = initial_sol(graph=graph, places=places)
print(init_sol.visited)

[0, 14, 11, 2, 12, 13, 3]


Pour définir une VNS, il faut définir les $k_\textrm{max}$ voisinages de recherche locale possibles. Pour notre problème, une bonne et simple répartition des voisinages est telle qu’un voisinage $k$ correspond à la permutation de $k$-paires de sommets dans $V(S)$.

On appelle **shaking** l’étape de génération d’une solution dans le voisinage $k$. Le travail qui suit correspond à l’implantation de cette étape. $\texttt{shaking}$ admet 3 arguments que sont la solution de départ, l’indice du voisinage $k$ ainsi que le graph courant.

Attention, avant d’implanter $\texttt{shaking}$, il est nécessaire de créer une méthode $\texttt{swap}$ dans la classe $\texttt{Solution}$. Cette méthode permet de mettre en œuvre la permutation dans une solution donnée (en mettant à jour tous les attributs nécessaires pour que la solution soit cohérente).

**Prescriptions d’implantation de shaking :**
- Sélectionner au hasard deux indices $i$ et $j$ différents et tels que $i, j \in \{2,...,m-1\}$
- Faire une copie de la solution courante et faire la permutation
- Retourner la solution créée

In [602]:
def shaking(sol, k):
    """
    Returns a solution on the k-th neighrboohood of sol
    """
    len_places = len(sol.visited)
    new_sol = copy.deepcopy(sol)
    for i in range(k):
        idx_1 = randint(1, len_places-2) # last number not included in randint (high exclusive)
        idx_2 = randint(1, len_places-2)
        while idx_1 == idx_2:
            idx_2 = randint(1, len_places-2)
        new_sol.swap(idx_1,idx_2)
    return new_sol

In [603]:
init_sol = initial_sol(graph=graph, places=places)
new_sol = shaking(init_sol, 10)

Une dernière étape essentielle dans une VNS est l’application d’un algorithme de recherche locale à la solution issue du shaking. Pour cela, on propose la recherche locale 2-opt. Celle-ci intervertit deux arcs dans la solution, à la recherche d’une qui est meilleure.

Pour un sommet $ i $, soit $ i '$ le successeur immédiat de $ i $ dans la séquence $ V (S) $. L'algorithme 2-opt fonctionne comme suit: pour chaque paire de sommets non consécutifs $ i, j $, vérifiez si en échangeant la position des sommets $ i '$ et $ j $ entraîne une amélioration du coût de la solution. Si oui, effectuez cet échange. Ce processus se répète jusqu'à ce qu'il n'y ait plus d'échanges rentables. On réalise cette opération pour toutes les paires d’arcs éligibles à la recherche du plus petit coût.

<img src="images/2opt.png" alt="" width="800"/>

<img src="images/2opt2.png" alt="" width="800"/>


Implantez $\texttt{local}\_\texttt{search}\_\texttt{2opt}$. 

**Prescriptions d’implantation :**
- Considérer chaque paire d’indices $i = \{2,..,m-3\}$ and $j = \{i+2, m-1\}$
- Si l’échange donne un plus bas coût, on le réalise
- Répéter jusqu’à optimum local.

In [604]:
def local_search_2opt(sol):
    """
    Apply 2-opt local search over sol
    """
    len_places = len(sol.visited)
    new_sol = copy.deepcopy(sol)
    price = sol.g
    for i in range(1, len_places-3):
        for j in range(i+2, len_places-1):
            new_sol.swap(i,j)
            if new_sol < price:
                price = new_sol.g
            else:
                new_sol.swap(i,j)
    if sol < new_sol:
        return sol
    return new_sol
            


In [605]:
print(init_sol.g)
init_sol.swap(1,3)
print(init_sol.g)
init_sol.swap(1,3)
print(init_sol.g)
print('\n')

init_sol = initial_sol(graph=graph, places=places)
new_sol = shaking(init_sol, 10)
new_sol = local_search_2opt(init_sol)
print(init_sol.g)
print(new_sol.g)

99
99
99


111
46


Finalement, il est temps d'implanter notre VNS. La méthode $\texttt{vns}$ reçoit une solution complète, le graphe courant, le nombre maximal de voisinages et un temps de calcul limite. Celle-ci retourne la solution optimale trouvée

**Prescriptions d’implantation :**
- À chaque itération, la VNS génère une solution dans le k-ème voisinage (shaking) à partir de la meilleure solution courante et applique une recherche locale 2-opt dessus
- Si la nouvelle solution trouvée a un meilleur coût, mettre à jour la meilleure solution courante
- Répéter le processus jusqu'à $\texttt{t}\_\texttt{max}$

In [755]:
import time


import time

t_end = time.time() + 60 * 15
    # do whatever you do
def vns(sol, k_max, t_max):
    """
    Performs the VNS algorithm
    """
    best_sol = copy.deepcopy(sol)
    t_end = time.time() + t_max

    while time.time() < t_end:
        new_sol = copy.deepcopy(shaking(best_sol, k_max))
        temp_sol = copy.deepcopy(local_search_2opt(new_sol))
        if temp_sol < best_sol:
            best_sol = copy.deepcopy(temp_sol)
    return best_sol
        

### 3.2 Experiments

Mettez en oeuvre la VNS sur les exemples d'illustration suivants et raportez les solutions obtenue:

In [756]:
# test 1  --------------  OPT. SOL. = 27
places=[0, 5, 13, 16, 6, 9, 4]
sol = initial_sol(graph=graph, places=places)
start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

27
[0, 5, 13, 16, 6, 9, 4]
--- 1.0007188320159912 seconds ---


In [757]:
#test 2  --------------  OPT. SOL. = 30
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)

print("--- %s seconds ---" % (time.time() - start_time))

30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 1.0005247592926025 seconds ---


In [758]:
# test 3  --------------  OPT. SOL. = 26
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 1.0004215240478516 seconds ---


In [763]:
# test 4  --------------  OPT. SOL. = 40
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

40
[0, 3, 9, 13, 15, 18, 20, 16, 11, 12, 14, 5, 4, 2, 1]
--- 1.0004258155822754 seconds ---


## 4. BONUS (1 point)

Expliquez dans quelle situation chacun des algorithmes développés est plus approprié (prenez en compte l’évolutivité du problème)

In [ ]:
#bfs poids egaux
#A* poids non egaux
#genitique? quand sastifait optimal local solution rapide